In [1]:
# let2int(): maps lower-case letters to corresponding integer number
def let2int(char):
    return ord(char) - ord('a')

In [2]:
# int2let(): maps intergers between 0...25 to 'a'...'z'
def int2let(n):
    return chr(ord('a') + n)

In [3]:
# isLower(): Determineswhether a letter is lower-case
def isLower(char):
    return ord(char) >= 97 and ord(char) <= 122

In [4]:
# shift(): shift a char over n positions
def shift(n, char):
    if isLower(char):
        return int2let((let2int(char) + n) % 26)
    else:
        return char

In [5]:
# encode(): encode xs shifting each letter n positions
def encode(n, xs):
    return [shift(n, x) for x in xs]

In [6]:
"".join(encode(7, "taking up an encoding that should be taken up by one of the candidates"))

'ahrpun bw hu lujvkpun aoha zovbsk il ahrlu bw if vul vm aol jhukpkhalz'

### Using brute force

In [7]:
std = 'ahrpun bw hu lujvkpun aoha zovbsk il ahrlu bw if vul vm aol jhukpkhalz'

def crack(decode):
    result = []
    i = 1
    while (i < 26):
        result.append(str(i) + ": " + "".join(encode(i, decode)))
        i = i + 1
    print(result)
    
crack(std)

['1: bisqvo cx iv mvkwlqvo bpib apwctl jm bismv cx jg wvm wn bpm kivlqlibma', '2: cjtrwp dy jw nwlxmrwp cqjc bqxdum kn cjtnw dy kh xwn xo cqn ljwmrmjcnb', '3: dkusxq ez kx oxmynsxq drkd cryevn lo dkuox ez li yxo yp dro mkxnsnkdoc', '4: elvtyr fa ly pynzotyr esle dszfwo mp elvpy fa mj zyp zq esp nlyotolepd', '5: fmwuzs gb mz qzoapuzs ftmf etagxp nq fmwqz gb nk azq ar ftq omzpupmfqe', '6: gnxvat hc na rapbqvat gung fubhyq or gnxra hc ol bar bs gur pnaqvqngrf', '7: hoywbu id ob sbqcrwbu hvoh gvcizr ps hoysb id pm cbs ct hvs qobrwrohsg', '8: ipzxcv je pc tcrdsxcv iwpi hwdjas qt ipztc je qn dct du iwt rpcsxspith', '9: jqaydw kf qd udsetydw jxqj ixekbt ru jqaud kf ro edu ev jxu sqdtytqjui', '10: krbzex lg re vetfuzex kyrk jyflcu sv krbve lg sp fev fw kyv treuzurkvj', '11: lscafy mh sf wfugvafy lzsl kzgmdv tw lscwf mh tq gfw gx lzw usfvavslwk', '12: mtdbgz ni tg xgvhwbgz matm lahnew ux mtdxg ni ur hgx hy max vtgwbwtmxl', '13: nuecha oj uh yhwixcha nbun mbiofx vy nueyh oj vs ihy iz nby wuhxcxu

Because new items are appended to the front of a list in Python, we have to read the list backwards from the end. We find our decoded sentence at number 19: 26 - 19 = 7, so 7 was the shift that was used to encode the string.

### Using our brain

We could leave it at this, because, as we will see, even if we will use our brain and try to come up with a clever solution, we will have to use a similar 'cycling' routine.

On the other hand, we might learmn something taking the clever route.

#### Letter frequency

Our clever approach is built on the observation that not all letters are used in the same frequency in a language. Using an observed frequency distribution of letters making up words in sentences (in a particular language), we can derive a table like the one below.

In [8]:
table = [
    8.1, 1.5, 2.8, 4.2, 12.7, 2.2, 2.0, 6.1, 7.0, 0.2,
    0.8, 4.0, 2.4, 6.7, 7.5, 1.9, 0.1, 6.0, 6.3, 9.0,
    2.8, 1.0, 2.4, 0.2, 2.0, 0.1
]
len(table)

26

The table above is the frequency table for letters in the English language. The letter 'e' appears the most, 12.7% frequency; 'z' and 'q' occur least often, both with a frequency of 0.1%.

We need some of the helper functions we defined above, when attacking the cipher "brute force":

- let2int()
- int2let()
- isLower()
- shift()
- encode()

We can also produce a frequency table of letters in an individual string. In order to do this we need some helper functions:

- count (counts the occurences of a letter in a string)

to define a function freqs() that returns the frequency table for any given string.

Given the above frequency table, we can also count the freq of of letters in an individual string. In order to do this we need some helper functions:

- percent()
- lowers()
- count(): counts the occurences of a letter in a string
- freqs

In [9]:
# percent() given the input of two integers it calculates the percentage of one integer
# with respect to the other
# percent(5, 15) should return 33.333
def percent(n,m):
    return float(n) / float(m) * 100
percent(5, 15)

33.33333333333333

In [10]:
str1 = 'aaaabbbbbbccccccccddd'
def lowers(xs):
    return len([x for x in xs if isLower(x)])
lowers(str1)

21

In [11]:
# count() given input letter return the frequency of that letter in the given string
def count(x, xs):
    return len([x_ for x_ in xs if x_ == x]) # Neat little trick
count('c', str1)

8

In [35]:
def freqs(xs):
    alphabet = [chr(o) for o in range(97, 123)]
    n = lowers(xs)
    return [percent(count(x, xs), n) for x in alphabet]

We now can use the two tables:

- table which gives us the expected frequency of letters (in English);
- freqs that gives the actual frequency of letters used in the sentence we want to decode (not having access to the shift that was used to encode the letters).

There are several ways to relate both tables with frequencies. Here we can use the *chi-square statistic* to calculate the smallest values for to given lists. But first two helpers:

- positions()
- rotate()

In [36]:
def positions(x, xs):
    return [i for x_, i in zip(xs, range(0, len(xs))) if x_ == x]
positions('a', str1)

[0, 1, 2, 3]

The next step we need is to cycle through the possible shifts in order to calculate the *chi-square* score for each possible shift (0-25). Basically the same idea we used in our brute force attack!

For that we need a function rotate.

In [14]:
# rotate(): rotates a list anti-clockwise
def rotate(n, xs):
    return xs[n:] + xs[:n]
rotate(4, str1)

'bbbbbbccccccccdddaaaa'

In [37]:
# chi square statistic http://en.wikipedia.org/wiki/Chi-squared_statisti
def chisqr(os, es):
    return sum([((o-e)*(o-e)) / e for o, e in zip(os, es)])

In [21]:
# main function to crack the encoded message
def crack(xs):
    distrib = freqs(xs)
    chitab = [chisqr(rotate(n, distrib), table) for n in range(0, 26)]
    factor = positions(min(chitab), chitab)[0]
    return encode(-factor, xs)

In [38]:
# Run the thing:
plaintext = "taking up an encoding that should be taken up by one of the candidates"
key = 7
ciphertext = encode(key, plaintext)
print("Encoded: %s" % "".join(ciphertext))
print("Decoded: %s" % "".join(crack(ciphertext)))

Encoded: ahrpun bw hu lujvkpun aoha zovbsk il ahrlu bw if vul vm aol jhukpkhalz
Decoded: taking up an encoding that should be taken up by one of the candidates
